In [3]:
#Phần này chỉ để chạy cho vui, không có ý nghĩa nhiều
import aiohttp
import asyncio
from bs4 import BeautifulSoup

In [4]:
async def crawl_page(session, page):
    url = f"https://www.topcv.vn/viec-lam-it?page={page}"
    async with session.get(url) as response:
        if response.status == 200:
            html = await response.text()
            soup = BeautifulSoup(html, "html.parser")
            all_job = soup.find_all("div", class_="job-item-2")
            links = [job.find("a", href=True)["href"] for job in all_job if job.find("a", href=True)]
            print(f"✅ Trang {page}: {len(links)} links")
            return links
        else:
            print(f"⚠️ Trang {page}: lỗi {response.status}")
            return []

In [8]:
async def main():
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/128.0.0.0 Safari/537.36"
    }
    async with aiohttp.ClientSession(headers=headers) as session:
        tasks = [crawl_page(session, page) for page in range(1, 45)]
        results = await asyncio.gather(*tasks)
        job_links = [link for sublist in results for link in sublist]
        print("\nTổng số link:", len(job_links))

In [9]:
# Chạy async event loop
await main()


⚠️ Trang 30: lỗi 429
⚠️ Trang 28: lỗi 429
⚠️ Trang 36: lỗi 429
⚠️ Trang 9: lỗi 429
⚠️ Trang 39: lỗi 429
⚠️ Trang 37: lỗi 429
⚠️ Trang 29: lỗi 429
⚠️ Trang 25: lỗi 429
⚠️ Trang 43: lỗi 429
⚠️ Trang 14: lỗi 429
⚠️ Trang 26: lỗi 429
⚠️ Trang 8: lỗi 429
⚠️ Trang 40: lỗi 429
⚠️ Trang 1: lỗi 429
✅ Trang 15: 50 links
⚠️ Trang 42: lỗi 429
⚠️ Trang 20: lỗi 429
⚠️ Trang 32: lỗi 429
⚠️ Trang 18: lỗi 429
✅ Trang 3: 50 links
⚠️ Trang 10: lỗi 429
⚠️ Trang 6: lỗi 429
⚠️ Trang 33: lỗi 429
✅ Trang 12: 50 links
⚠️ Trang 4: lỗi 429
✅ Trang 11: 50 links
✅ Trang 13: 50 links
✅ Trang 2: 50 links
⚠️ Trang 5: lỗi 429
✅ Trang 19: 50 links
✅ Trang 31: 50 links
✅ Trang 24: 50 links
✅ Trang 17: 50 links
✅ Trang 41: 50 links
✅ Trang 34: 50 links
✅ Trang 22: 50 links
✅ Trang 23: 50 links
✅ Trang 44: 6 links
✅ Trang 27: 50 links
✅ Trang 16: 50 links
✅ Trang 21: 50 links
✅ Trang 7: 50 links
✅ Trang 35: 50 links
✅ Trang 38: 50 links

Tổng số link: 1006
